# 🎯 Shot Gather 생성 및 노이즈 제거 워크플로우
# Shot Gather Generation & Denoising Workflow

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/knocgp/seismic/blob/main/Shot_Gather_Workflow.ipynb)

---

## 📋 워크플로우

1. ✅ **랜덤 합성 모델 생성** - 완전 랜덤 지층 모델
2. ✅ **Shot Gather 생성** - 다중 트레이스 탄성파 데이터
3. ✅ **노이즈 추가** - 실제 현장 노이즈 시뮬레이션
4. ✅ **노이즈 제거** - F-K 필터, 밴드패스, Median 필터
5. ✅ **비교 및 다운로드** - 전체 비교 및 NPZ 파일 다운로드

---

## 🚀 사용법

**"런타임" > "모두 실행"** 클릭 → 자동으로 전체 워크플로우 실행!

## 📦 1. 패키지 설치

In [ ]:
!pip install -q numpy scipy matplotlib

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import median_filter
import warnings
warnings.filterwarnings('ignore')

print("✅ 패키지 설치 완료!")

## 📥 2. GitHub 코드 다운로드 (Option 1)

In [ ]:
# GitHub에서 코드 다운로드
!git clone https://github.com/knocgp/seismic.git
%cd seismic

print("✅ 코드 다운로드 완료!")

## 🎬 3. 전체 워크플로우 실행

In [ ]:
# 전체 워크플로우 실행
!python shot_gather_workflow.py

## 💾 4. 결과 파일 다운로드

In [ ]:
from google.colab import files

# 생성된 파일 목록 확인
print("📂 생성된 파일:")
!ls -lh shot_gather_*.npz

print("\n다운로드 시작...")

# 1. 원본 Shot Gather
files.download('shot_gather_clean.npz')
print("✓ shot_gather_clean.npz 다운로드 완료")

# 2. 노이즈 추가된 Shot Gather
files.download('shot_gather_noisy.npz')
print("✓ shot_gather_noisy.npz 다운로드 완료")

# 3. 노이즈 제거된 Shot Gather
files.download('shot_gather_denoised.npz')
print("✓ shot_gather_denoised.npz 다운로드 완료")

print("\n✅ 모든 파일 다운로드 완료!")

## ⚙️ 5. 커스텀 파라미터로 재실행 (선택사항)

파라미터를 변경하여 다른 결과를 생성할 수 있습니다!

In [ ]:
# 코드를 직접 임포트하여 실행
import sys
sys.path.insert(0, '/content/seismic')

from shot_gather_workflow import ShotGatherProcessor

# 🎯 커스텀 파라미터
N_TRACES = 60           # 트레이스 개수
OFFSET_MIN = 50         # 최소 오프셋 (m)
OFFSET_MAX = 3000       # 최대 오프셋 (m)
WAVELET_FREQ = 30.0     # Wavelet 주파수 (Hz)
NOISE_LEVEL = 0.15      # 노이즈 레벨 (0-1)
N_LAYERS = 7            # 지층 개수

print("🎯 커스텀 파라미터:")
print(f"  트레이스 개수: {N_TRACES}")
print(f"  오프셋 범위: {OFFSET_MIN}-{OFFSET_MAX} m")
print(f"  Wavelet 주파수: {WAVELET_FREQ} Hz")
print(f"  노이즈 레벨: {NOISE_LEVEL}")
print(f"  지층 개수: {N_LAYERS}")
print()

# 프로세서 생성
processor = ShotGatherProcessor(dt=0.002, nt=1500)

# 1. 모델 생성
print("[1] 랜덤 모델 생성...")
model = processor.create_random_model(nlayers=N_LAYERS)
processor.plot_model(model)

# 2. Shot Gather 생성
print("[2] Shot Gather 생성...")
clean, offsets = processor.generate_shot_gather(
    model, n_traces=N_TRACES,
    offset_min=OFFSET_MIN, offset_max=OFFSET_MAX,
    freq=WAVELET_FREQ
)
processor.plot_shot_gather(clean, offsets, "Custom Clean Shot Gather")

# 3. 노이즈 추가
print("[3] 노이즈 추가...")
noisy = processor.add_realistic_noise(clean, noise_level=NOISE_LEVEL)
processor.plot_shot_gather(noisy, offsets, "Custom Noisy Shot Gather")

# 4. 노이즈 제거
print("[4] 노이즈 제거...")
denoised = processor.denoise_combined(noisy)
processor.plot_shot_gather(denoised, offsets, "Custom Denoised Shot Gather")

# 5. 비교
print("[5] 전체 비교...")
processor.plot_comparison(clean, noisy, denoised, offsets)

# 6. 저장
print("[6] 저장...")
np.savez('custom_shot_clean.npz', shot_gather=clean, offsets=offsets, time=processor.time, model=model)
np.savez('custom_shot_noisy.npz', shot_gather=noisy, offsets=offsets, time=processor.time, model=model)
np.savez('custom_shot_denoised.npz', shot_gather=denoised, offsets=offsets, time=processor.time, model=model)

print("\n✅ 커스텀 워크플로우 완료!")

## 📊 6. 저장된 데이터 로드 및 분석

In [ ]:
# 저장된 데이터 로드
clean_data = np.load('shot_gather_clean.npz', allow_pickle=True)
noisy_data = np.load('shot_gather_noisy.npz', allow_pickle=True)
denoised_data = np.load('shot_gather_denoised.npz', allow_pickle=True)

# 데이터 추출
clean_shot = clean_data['shot_gather']
noisy_shot = noisy_data['shot_gather']
denoised_shot = denoised_data['shot_gather']
offsets = clean_data['offsets']
time = clean_data['time']
model = clean_data['model'].item()

print("📊 데이터 정보:")
print(f"  Shot Gather 크기: {clean_shot.shape}")
print(f"  트레이스 개수: {clean_shot.shape[1]}")
print(f"  시간 샘플: {clean_shot.shape[0]}")
print(f"  샘플링 간격: {(time[1]-time[0])*1000:.1f} ms")
print(f"  총 시간: {time[-1]:.2f} s")
print(f"  오프셋 범위: {offsets[0]:.0f} - {offsets[-1]:.0f} m")
print(f"  지층 개수: {len(model['name'])}")

# 통계
print("\n📈 통계:")
print(f"  Clean RMS: {np.sqrt(np.mean(clean_shot**2)):.6f}")
print(f"  Noisy RMS: {np.sqrt(np.mean(noisy_shot**2)):.6f}")
print(f"  Denoised RMS: {np.sqrt(np.mean(denoised_shot**2)):.6f}")

# SNR 계산
noise = noisy_shot - clean_shot
residual = denoised_shot - clean_shot
snr_before = 20 * np.log10(np.std(clean_shot) / np.std(noise))
snr_after = 20 * np.log10(np.std(clean_shot) / np.std(residual))

print(f"\n  SNR (노이즈 추가 후): {snr_before:.2f} dB")
print(f"  SNR (노이즈 제거 후): {snr_after:.2f} dB")
print(f"  SNR 개선: {snr_after - snr_before:.2f} dB")

print("\n✅ 데이터 로드 완료!")

## 🔬 7. 추가 분석 - 단일 트레이스 비교

In [ ]:
# 중간 오프셋 트레이스 선택
trace_idx = len(offsets) // 2

# 플롯
fig, axes = plt.subplots(1, 3, figsize=(18, 8))

axes[0].plot(clean_shot[:, trace_idx], time, 'b-', linewidth=1.5)
axes[0].set_xlabel('Amplitude', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Time (s)', fontsize=12, fontweight='bold')
axes[0].set_title(f'Clean Trace (Offset={offsets[trace_idx]:.0f}m)', 
                  fontsize=13, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(True, alpha=0.3)

axes[1].plot(noisy_shot[:, trace_idx], time, 'r-', linewidth=1.5)
axes[1].set_xlabel('Amplitude', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Time (s)', fontsize=12, fontweight='bold')
axes[1].set_title(f'Noisy Trace (Offset={offsets[trace_idx]:.0f}m)', 
                  fontsize=13, fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(True, alpha=0.3)

axes[2].plot(denoised_shot[:, trace_idx], time, 'g-', linewidth=1.5)
axes[2].set_xlabel('Amplitude', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Time (s)', fontsize=12, fontweight='bold')
axes[2].set_title(f'Denoised Trace (Offset={offsets[trace_idx]:.0f}m)', 
                  fontsize=13, fontweight='bold')
axes[2].invert_yaxis()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✅ 트레이스 {trace_idx} (오프셋 {offsets[trace_idx]:.0f}m) 비교 완료!")

## 📊 8. 주파수 스펙트럼 분석

In [ ]:
# 중간 트레이스 FFT
trace_idx = len(offsets) // 2
dt = time[1] - time[0]

# FFT 계산
freq = np.fft.rfftfreq(len(time), dt)
clean_fft = np.abs(np.fft.rfft(clean_shot[:, trace_idx]))
noisy_fft = np.abs(np.fft.rfft(noisy_shot[:, trace_idx]))
denoised_fft = np.abs(np.fft.rfft(denoised_shot[:, trace_idx]))

# 플롯
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(freq, clean_fft, 'b-', linewidth=2, label='Clean', alpha=0.8)
ax.plot(freq, noisy_fft, 'r-', linewidth=2, label='Noisy', alpha=0.6)
ax.plot(freq, denoised_fft, 'g-', linewidth=2, label='Denoised', alpha=0.8)

ax.set_xlabel('Frequency (Hz)', fontsize=13, fontweight='bold')
ax.set_ylabel('Amplitude Spectrum', fontsize=13, fontweight='bold')
ax.set_title(f'Frequency Spectrum Comparison (Offset={offsets[trace_idx]:.0f}m)', 
             fontsize=14, fontweight='bold')
ax.set_xlim([0, 100])
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ 주파수 스펙트럼 분석 완료!")

## 📚 참고 정보

### 📁 생성되는 파일

1. **shot_gather_clean.npz** - 원본 Shot Gather
2. **shot_gather_noisy.npz** - 노이즈 추가된 Shot Gather
3. **shot_gather_denoised.npz** - 노이즈 제거된 Shot Gather

### 🔧 노이즈 제거 기법

1. **F-K 필터**
   - Frequency-Wavenumber 도메인 필터링
   - Ground Roll (저속 표면파) 제거
   - 속도 기반 필터링

2. **밴드패스 필터**
   - 8-60 Hz 범위 통과
   - 저주파 노이즈 제거
   - 고주파 노이즈 제거

3. **Median 필터**
   - 스파이크 노이즈 제거
   - Bad trace 보정
   - 비선형 필터링

### 🎯 주요 파라미터

- **n_traces**: 트레이스 개수 (기본 48)
- **offset_min/max**: 오프셋 범위 (m)
- **freq**: Wavelet 주파수 (Hz)
- **noise_level**: 노이즈 레벨 (0-1)
- **nlayers**: 지층 개수

### 🔗 GitHub 저장소

https://github.com/knocgp/seismic

---

**Made with ❤️ for Seismic Data Processing**